<a href="https://colab.research.google.com/github/Artorias1469/Neural_networks_1/blob/main/%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D1%8B%D0%B9_%D1%81%D0%BB%D0%BE%D0%B9_(Dense)_%D0%94%D0%97_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

Всего должно получиться 18 комбинаций указанных параметров.

Создайте сравнительную таблицу по результатам проведенных тестов.

In [1]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [2]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [3]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_light.zip

In [4]:
# Путь к директории с базой
base_dir = '/content/hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else:
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

x_train = x_train / 255.0
y_train = to_categorical(y_train, 3)

neurons_list = [10, 100, 5000]
activations = ['relu', 'linear']
batch_sizes = [10, 100, 1000]

results = []

for neurons in neurons_list:
    for activation in activations:
        for batch_size in batch_sizes:
            model = Sequential([
                Flatten(input_shape=(20, 20, 1)),
                Dense(neurons, activation=activation),
                Dense(3, activation='softmax')
            ])

            model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

            history = model.fit(x_train, y_train, epochs=10, batch_size=batch_size, verbose=0)

            accuracy = history.history['accuracy'][-1]

            results.append({'Neurons': neurons, 'Activation': activation, 'Batch Size': batch_size, 'Accuracy': accuracy})

            print(f'Neurons: {neurons}, Activation: {activation}, Batch Size: {batch_size}, Accuracy: {accuracy:.4f}')

df_results = pd.DataFrame(results)
print(df_results)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Neurons: 10, Activation: relu, Batch Size: 10, Accuracy: 0.8344
Neurons: 10, Activation: relu, Batch Size: 100, Accuracy: 0.6556
Neurons: 10, Activation: relu, Batch Size: 1000, Accuracy: 0.4106
Neurons: 10, Activation: linear, Batch Size: 10, Accuracy: 0.7914
Neurons: 10, Activation: linear, Batch Size: 100, Accuracy: 0.5331
Neurons: 10, Activation: linear, Batch Size: 1000, Accuracy: 0.4702
Neurons: 100, Activation: relu, Batch Size: 10, Accuracy: 0.8974
Neurons: 100, Activation: relu, Batch Size: 100, Accuracy: 0.6589
Neurons: 100, Activation: relu, Batch Size: 1000, Accuracy: 0.6623
Neurons: 100, Activation: linear, Batch Size: 10, Accuracy: 0.8245
Neurons: 100, Activation: linear, Batch Size: 100, Accuracy: 0.6589
Neurons: 100, Activation: linear, Batch Size: 1000, Accuracy: 0.5397
Neurons: 5000, Activation: relu, Batch Size: 10, Accuracy: 0.8742
Neurons: 5000, Activation: relu, Batch Size: 100, Accuracy: 0.7748
Neurons: 5000, Activation: relu, Batch Size: 1000, Accuracy: 0.5894
N